In [1]:
from tqdm.auto import tqdm
import pandas as pd
tqdm.pandas()

In [2]:
df = pd.read_csv("../data/test.csv")
df = df.rename(columns={"c_text": "comment_text"})
df

,comment_id,comment_text
0,3245,@USER Sie würden wahrscheinlich auch einen Kri...
1,3246,"@USER , ich glaube,Sie verkrnnen gründlich die..."
2,3247,@USER: Man kann natürlich immerzu dieselbe Sau...
3,3248,"Als jemand, der im real existierenden Sozialis..."
4,3249,Ihr größter Erfolg in ihrem Leben? KLASSENSPRE...
...,...,...
939,4184,@USER Da ist wie gegen Windmühlen kämpfen
940,4185,@USER Ihr Wort in Gottes Ohr.
941,4186,😂
942,4187,Mimimimi


In [3]:
with open("../data/emojis_dict.json") as f:
  import json
  emoji_dict=json.load(f)

In [4]:
import re
from nltk.corpus import stopwords
df['user']=df['comment_text'].apply(lambda comment: 1 if "@USER" in comment else 0)
df['mention']=df['comment_text'].apply(lambda comment: 1 if "@MEDIUM" in comment else 0)
df['mod']=df['comment_text'].apply(lambda comment: 1 if "@MODERATOR" in comment else 0)
df['total_length'] = df['comment_text'].apply(len)
df['num_words'] = df.comment_text.str.count('\S+')
df['all_caps']= df['comment_text'].apply(lambda comment: sum(1 for c in comment.split() if c.isupper() and c not in ["@USER", "@MEDIUM", "@MODERATOR"]))/df['num_words']
df['emoji']=df['comment_text'].apply(lambda comment: len(re.findall(u'[\U0001f600-\U0001f650]', comment)))/df['num_words']
df['caps']= df['all_caps'].apply(lambda num: 1 if num > 0 else 0 )
df['capitals'] = df['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
df['caps_vs_length'] = df.apply(lambda row: float(row['capitals'])/float(row['total_length']),
                                axis=1)
df['num_unique_words'] = df['comment_text'].apply(lambda comment: len(set(w for w in comment.split())))
df['words_vs_unique'] = df['num_unique_words'] / df['num_words']
df['num_urls']=df['comment_text'].apply(lambda comment: len(re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', comment)))
df['!']=df['comment_text'].apply(lambda comment : comment.count("!"))/df['num_words']
df['?']=df['comment_text'].apply(lambda comment : comment.count('?'))/df['num_words']
import string
c=0
def transcribe(text):
  global c
  new_text=text
  for emoji,de_text in emoji_dict.items():
    new_text=new_text.replace(emoji," "+de_text+" ")
    if new_text!=text:
      c+=1
  return new_text
df['punc'] = df["comment_text"].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))/df['num_words']
df["text"]= df['comment_text'].progress_apply(lambda x: transcribe(x))
print(c)
df["text"]= df['text'].map(lambda x: x.replace("@USER",""))
df["text"]= df['text'].map(lambda x: x.replace("@MEDIUM",""))
df["text"]= df['text'].map(lambda x: x.replace("@MODERATOR",""))
df['text'] = df['text'].map(lambda x: re.sub('\\n',' ',str(x)))
# remove IP addresses or user IDs
df['text'] = df['text'].map(lambda x: re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",'',str(x)))
# lower uppercase letters
df['text'] = df['text'].map(lambda x: str(x).lower())
#remove http links in the text
df['text'] = df['text'].map(lambda x: re.sub("(http://.*?\s)|(http://.*)",'',str(x)))
#remove all punctuation except for apostrophe (')
df['text'] = df['text'].map(lambda x: re.sub('[!"#$%&\()*+,-/:;<=>?@[\\]^_`{|}~]','',str(x)))
df['stop']= df["text"].progress_apply(lambda comment: sum(1 for w in comment.split() if w in stopwords.words('german')))/df['num_words']

  0%|          | 0/944 [00:00<?, ?it/s]

584501


  0%|          | 0/944 [00:00<?, ?it/s]

In [5]:
df

,comment_id,comment_text,user,mention,mod,total_length,num_words,all_caps,emoji,caps,capitals,caps_vs_length,num_unique_words,words_vs_unique,num_urls,!,?,punc,text,stop
0,3245,@USER Sie würden wahrscheinlich auch einen Kri...,1,0,0,159,25,0.000000,0.0,0,9,0.056604,24,0.960000,0,0.000000,0.000000,0.320000,sie würden wahrscheinlich auch einen kriegstr...,0.440000
1,3246,"@USER , ich glaube,Sie verkrnnen gründlich die...",1,0,0,187,29,0.000000,0.0,0,11,0.058824,26,0.896552,0,0.000000,0.000000,0.206897,ich glaubesie verkrnnen gründlich die situat...,0.482759
2,3247,@USER: Man kann natürlich immerzu dieselbe Sau...,1,0,0,840,120,0.041667,0.0,1,48,0.057143,100,0.833333,1,0.000000,0.000000,0.291667,man kann natürlich immerzu dieselbe sau durch...,0.458333
3,3248,"Als jemand, der im real existierenden Sozialis...",0,0,0,332,55,0.000000,0.0,0,11,0.033133,50,0.909091,0,0.000000,0.000000,0.272727,als jemand der im real existierenden sozialism...,0.563636
4,3249,Ihr größter Erfolg in ihrem Leben? KLASSENSPRE...,0,0,0,53,7,0.142857,0.0,1,20,0.377358,7,1.000000,0,0.142857,0.142857,0.285714,ihr größter erfolg in ihrem leben klassensprec...,0.428571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,4184,@USER Da ist wie gegen Windmühlen kämpfen,1,0,0,41,7,0.000000,0.0,0,6,0.146341,7,1.000000,0,0.000000,0.000000,0.142857,da ist wie gegen windmühlen kämpfen,0.571429
940,4185,@USER Ihr Wort in Gottes Ohr.,1,0,0,29,6,0.000000,0.0,0,8,0.275862,6,1.000000,0,0.000000,0.000000,0.333333,ihr wort in gottes ohr,0.333333
941,4186,😂,0,0,0,1,1,0.000000,1.0,0,0,0.000000,1,1.000000,0,0.000000,0.000000,0.000000,gesicht mit freudentränen,1.000000
942,4187,Mimimimi,0,0,0,8,1,0.000000,0.0,0,1,0.125000,1,1.000000,0,0.000000,0.000000,0.000000,mimimimi,0.000000


In [6]:
import json
f1 = open("../data/positives.json")
f2 = open("../data/negatives.json")
positive = json.load(f1)
negative = json.load(f2)
positive_words = [i[0] for i in positive]
positive_scores = [i[1] for i in positive]
negative_words = [i[0] for i in negative]
negative_scores = [i[1] for i in negative]

pos_score = []
neg_score = []

def find_negative(text):
  words = text.split()
  count = 0
  score = 0
  for w in words:
    try:
      indx = negative_words.index(w.lower())
      count+=1
      score+= negative_scores[indx]
    except:
      pass
  neg_score.append(abs(score))
  return count

def find_positive(text):
  words = text.split()
  count = 0
  score = 0
  for w in words:
    try:
      indx = positive_words.index(w.lower())
      count+=1
      score+= positive_scores[indx]
    except:
      pass
  pos_score.append(abs(score))
  return count

df["num_negative_words"] = df["text"].progress_apply(lambda x: find_negative(x))
df["negativity_score"] = neg_score
df["num_positive_words"] = df["text"].progress_apply(lambda x: find_positive(x))
df["positivity_score"] = pos_score
df['neg']=df["negativity_score"]/df["num_negative_words"]
df['pos']=df["positivity_score"]/df["num_positive_words"]
df['pos']=df['pos'].fillna(0)
df['neg']=df['neg'].fillna(0)

  0%|          | 0/944 [00:00<?, ?it/s]

  0%|          | 0/944 [00:00<?, ?it/s]

In [7]:
import language_tool_python
tool = language_tool_python.LanguageToolPublicAPI('de')
match_u=[]
def get_errors(text):
  global match_u
  matches = tool.check(text)
  matches=[match for match in matches if match.ruleId not in ['DE_CASE','DE_DASH','DE_DU_UPPER_LOWER','DE_PHRASE_REPETITION','DE_SENTENCE_WHITESPACE','EMAIL','GERMAN_WORD_REPEAT_BEGINNING_RULE',
 'GERMAN_WORD_REPEAT_RULE','PLURAL_APOSTROPH','PUNCTUATION_PARAGRAPH_END','UNPAIRED_BRACKETS',
 'UPPERCASE_SENTENCE_START']]
  match_ui=[match.ruleId for match in matches]
  match_u.extend(match_ui)
  return len(matches)/len(text)
df['error']=df['comment_text'].progress_apply(lambda comment: get_errors(comment))

self._url: https://api.languagetool.org/v2/


  0%|          | 0/944 [00:00<?, ?it/s]

In [8]:
german = {'present':['nutze ab',
                      'nutzt ab',
                      'nutzen ab',
                      'gehe aus',
                      'gehst aus',
                      'geht aus',
                      'gehen aus',
                      'rufe an',
                      'rufst an',
                      'ruft an',
                      'rufen an',
                      'rufe an',
                      'rufst an',
                      'ruft an',
                      'rufen an',
                      'gehe auf',
                      'gehst auf',
                      'geht auf',
                      'gehen auf',
                      'breche zusammen',
                      'brichst zusammen',
                      'bricht zusammen',
                      'brechen zusammen',
                      'brecht zusammen',
                      'arbeite',
                      'arbeitest',
                      'arbeitet',
                      'arbeiten',
                      'bin',
                      'bist', 
                      'ist', 
                      'sind',#typo
                      'seid',#typo
                      'sein',
                      'komme',
                      'kommst',
                      'kommt',
                      'kommen',
                      'steige',
                      'steigst',
                      'steigt',
                      'steigen',
                      'bekomme',
                      'bekommst',
                      'bekommt',
                      'bekommen',
                      'mache',
                      'machst',
                      'macht',
                      'machen',
                      'nutzen',
                      'nutze',
                      'nutzt',
                      'habe',
                      'hast',
                      'hat',
                      'haben',
                      'habt',
                      'fühle',
                      'fühlst',
                      'fühlt',
                      'fühlen',
                      'fuehlst',
                      'gewinne',
                      'gewinnst',
                      'gewinnt',
                      'gewinnen',
                      'regne',
                      'regnest',
                      'regnet',
                      'regnen',
                      'kaufe',
                      'kaufst',
                      'kauft',
                      'kaufen',
                      'sterbe',
                      'stirbst',
                      'stirbt',
                      'sterben',
                      'sterbt',
                      'reise',
                      'reist',
                      'reisen',
                      "rijs",#typo
                      'ruft',
                      'rufst',
                      'rufe',
                      'rufen',
                      'gehe',
                      'gehst',
                      'geht',
                      'gehen',
                      'liebe',
                      'liebst',
                      'liebt',
                      'lieben',
                      'lebe', 
                      'lebst', 
                      'lebt', 
                      'leben'
            ],
    'future': 
            ['werde',
              'wirst',
              'wird',
              'werden',
              'werdet',
              'werden',
              'wirdt',#typo
              'werde',
              'wirst',
              'wird',
              'werden',
              'werdet',
              'werden',
              'will',
              'willst',
              'wollen',
              'wollt',
              'wollen',
              ##konjunktiv ii of mogen meaning to want/would like
              'möchte',
              'möchtest',
              'möchten',
              'möchtet',
              'hoffe',#hope
              'hoffst',
              'hofft',
              'hoffen',
              'plane',
              'planst',
              'plant',
              'plannen',
              ],
    'uncertainity':
                 [#indicative of können
                  'kann',
                  'kannst',
                  'könnt',
                  'können',
                  
                  #konjunktiv ii of können
                  'könnte',
                  'könntest',
                  'könnten',
                  'könntet',
                  
                  #may have some epistemic use, but not typical, as in english ''should' (Nuyts 2000). Both konjunctiv ii and indicative included    
                  'sollen',
                  'soll',
                  'sollst',
                  'sollt',
                  
                  #konjunktiv ii
                  'sollte',
                  'solltest',
                  'sollten',
                  'solltet',
                  
                  #indicative dürfen cannot have epistemic uses, only deontic (Nuyts, 2000)
                  #'darf',
                  #'darfst',
                  #'dürfen',
                  #'dürft',
                  
                  #konjunktiv ii of ¨dürfen
                  'dürfte',
                  'dürftest',
                  'dürften',
                  'dürftet',
                  
                  #mogen 'may' in the indicative has epistemic uses, but not konjunktiv ii (Nuyts 2000)
                  'mag',
                  'magst',
                  'mögen',
                  'mögt',
                  
                  
                  ##konjuntiv of werden, i.e. 'would' with epistemic uses: according to informant/coder
                  'würde',
                  'würdest',
                  'würden',
                  'würdet',
                  
                  #konjunktiv ii of müssen has epistemic uses, like 'should' (Mortelmans 2000).
                  'müßte',
                  'müßtest',
                  'müßten',
                  'müßtet',
                  'müsste',
                  'müsstest',
                  'müssten',
                  'müsstet',
                  'unter umständen',
                  'annehmbar',#presumably (Nuyts 2000)
                  'eventuell',
                  'anscheinend',
                  'gegebenenfalls',
                  'wahrscheinlichkeit',#probability -- informant coder
                  'möglich',
                  'möglicherweise',
                  'offenbar',
                  'scheinbar',#seemingly
                  #'vielleicht',
                  'vermutlich',#presumably (Nuyts 2000)
                  'wahrscheinlich',
                  'womöglich',
                  'wohl',
                  'vielleicht',
                  'aber',
                  'auch',
                  'bloß',
                  'denn',
                  'doch',
                  'eigentlich',
                  'eben',
                  'etwa',
                  'einfach',
                  'erst',
                  'halt',
                  'ja',
                  'nun',
                  'mal',
                  'nur',
                  'schon',
                  'vielleicht',
                  'ruhig'
                  ],#maybe
    
    'certainity':
                 ['muss',
                  'musst',
                  'müssen',
                  'müsst',
                  'muß',
                  'mußt',
                  'müßen',
                  'müßt',
                  'auf jeden fall',
                  'klipp und klar',
                  'aufjedenfall',
                  'augenscheinlich',#evidently
                  'bestimmt',#certainly
                  'definitiv',
                  'deutlich',#clearly
                  'eindeutig',
                  'gewiss',
                  'klar',
                  'offensichtlich',#obviously
                  'jedenfalls',
                  'sicher',#certainly (Nuyts 2000)
                  'sicherlich',#certainly
                  'zweifelsohne',#certainly
                  'zweifellos'#certainly
                  ],
    'mental':      ['nehme an',
                    'nimmst an',
                    'nimmt an',
                    'nehmen an',
                    'nehmt an', #assume, but with more clear qualificational use 
                    'denke',
                    'denkst',
                    'denkt',
                    'denken',
                    'glaube',
                    'glaubst',
                    'glaubt',
                    'glauben',# Nuyts 2000
                    'meine',
                    'meinst',
                    'meint',
                    'meinen', # to mean, no qualificational use in english, but stronger in german, and menen, too in dutch (Nuyts 2000)
                    'vermuten',#to presume (outdated in engish mostly, (Nuyts, 2000))
                    'vermute',
                    'vermutest',
                    'vermutet',
                    'rechne',#reckon, qulaificational use, (Nuyts 2000)
                    'rechnest',
                    'rechnet',
                    'rechnen',
                    #'sage',#while this can be used epistemically, none of the question frames do so
                    #'sagst',# and do use 'to say' in non epistemic ways
                    #'sagt',
                    #'sagen',
                    'erwarte',
                    'erwartest',
                    'erwartet',
                    'erwarten'
                  ],
     }
df['present']= df["text"].apply(lambda comment: sum(1 for w in comment.split() if w in german['present']))/df['num_words']
df['future']= df["text"].apply(lambda comment: sum(1 for w in comment.split() if w in german['future']))/df['num_words']
df['uncertainity']= df["text"].apply(lambda comment: sum(1 for w in comment.split() if w in german['uncertainity']))/df['num_words']
df['certainity']= df["text"].apply(lambda comment: sum(1 for w in comment.split() if w in german['certainity']))/df['num_words']
df['mental']= df["text"].apply(lambda comment: sum(1 for w in comment.split() if w in german['mental']))/df['num_words']
verbs=pd.read_csv("../data/top-german-verbs.csv")

In [9]:
df['Infinitiv']= df["text"].progress_apply(lambda comment: sum(1 for w in comment.split() if w in list(verbs['Infinitiv'])))/df['num_words']
df['Präsens_ich']= df["text"].progress_apply(lambda comment: sum(1 for w in comment.split() if w in list(verbs['Präsens_ich'])))/df['num_words']
df['Präsens_du']= df["text"].progress_apply(lambda comment: sum(1 for w in comment.split() if w in list(verbs['Präsens_du'])))/df['num_words']
df['Präsens_er, sie, es']= df["text"].progress_apply(lambda comment: sum(1 for w in comment.split() if w in list(verbs['Präsens_er, sie, es'])))/df['num_words']
df['Präteritum_ich']= df["text"].progress_apply(lambda comment: sum(1 for w in comment.split() if w in list(verbs['Präteritum_ich'])))/df['num_words']
df['Partizip II']= df["text"].progress_apply(lambda comment: sum(1 for w in comment.split() if w in list(verbs['Partizip II'])))/df['num_words']
df['Konjunktiv II_ich']= df["text"].progress_apply(lambda comment: sum(1 for w in comment.split() if w in list(verbs['Konjunktiv II_ich'])))/df['num_words']

  0%|          | 0/944 [00:00<?, ?it/s]

  0%|          | 0/944 [00:00<?, ?it/s]

  0%|          | 0/944 [00:00<?, ?it/s]

  0%|          | 0/944 [00:00<?, ?it/s]

  0%|          | 0/944 [00:00<?, ?it/s]

  0%|          | 0/944 [00:00<?, ?it/s]

  0%|          | 0/944 [00:00<?, ?it/s]

In [10]:
df.text.fillna(" ")
df.fillna(0)

,comment_id,comment_text,user,mention,mod,total_length,num_words,all_caps,emoji,caps,...,uncertainity,certainity,mental,Infinitiv,Präsens_ich,Präsens_du,"Präsens_er, sie, es",Präteritum_ich,Partizip II,Konjunktiv II_ich
0,3245,@USER Sie würden wahrscheinlich auch einen Kri...,1,0,0,159,25,0.000000,0.0,0,...,0.200000,0.000000,0.0,0.120000,0.000000,0.000000,0.040000,0.000000,0.000000,0.040000
1,3246,"@USER , ich glaube,Sie verkrnnen gründlich die...",1,0,0,187,29,0.000000,0.0,0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.137931,0.034483,0.034483,0.034483
2,3247,@USER: Man kann natürlich immerzu dieselbe Sau...,1,0,0,840,120,0.041667,0.0,1,...,0.083333,0.000000,0.0,0.083333,0.025000,0.008333,0.066667,0.008333,0.008333,0.008333
3,3248,"Als jemand, der im real existierenden Sozialis...",0,0,0,332,55,0.000000,0.0,0,...,0.109091,0.018182,0.0,0.018182,0.054545,0.000000,0.127273,0.018182,0.072727,0.000000
4,3249,Ihr größter Erfolg in ihrem Leben? KLASSENSPRE...,0,0,0,53,7,0.142857,0.0,1,...,0.000000,0.000000,0.0,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,4184,@USER Da ist wie gegen Windmühlen kämpfen,1,0,0,41,7,0.000000,0.0,0,...,0.000000,0.000000,0.0,0.142857,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000
940,4185,@USER Ihr Wort in Gottes Ohr.,1,0,0,29,6,0.000000,0.0,0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
941,4186,😂,0,0,0,1,1,0.000000,1.0,0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
942,4187,Mimimimi,0,0,0,8,1,0.000000,0.0,0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
import readability
def get_readability(text):
  try:
    results = readability.getmeasures(text, lang='de')
  except:
    print("error")
    return 0
  return results["readability grades"]["DaleChallIndex"]
df["readability"]=df["comment_text"].progress_apply(lambda x: get_readability(x))

  0%|          | 0/944 [00:00<?, ?it/s]

error
error


In [12]:
df.to_csv("../data/test_ft.csv")